In [1]:
import dask.dataframe as dd
import dask.array as da
import numpy as np
from dask.distributed import Client
client = Client(n_workers=4) #cuz four cores/threads
client

2023-05-15 14:34:30,904 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-r11ig_g3', purging
2023-05-15 14:34:30,906 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-e3xs7ftj', purging
2023-05-15 14:34:30,906 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ub_1mq5y', purging
2023-05-15 14:34:30,907 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vj2qn9c3', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 11.41 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38367,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 11.41 GiB
Comm: tcp://127.0.0.1:37641,Total threads: 1
Dashboard: http://127.0.0.1:40657/status,Memory: 2.85 GiB
Nanny: tcp://127.0.0.1:36279,


In [5]:
import os

# read the first file as dask dataframe
ddf = dd.read_csv(
    os.path.join("data", "nycflights", "*.csv"),
    parse_dates={"Date": [0, 1, 2]},
    dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
)



In [14]:
ddf.head(10)

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1990-01-01,1,1621.0,1540,1747.0,1701,US,33,NaN,86.0,...,NaN,46.0,41.0,EWR,PIT,319.0,NaN,NaN,False,0
1,1990-01-02,2,1547.0,1540,1700.0,1701,US,33,NaN,73.0,...,NaN,-1.0,7.0,EWR,PIT,319.0,NaN,NaN,False,0
2,1990-01-03,3,1546.0,1540,1710.0,1701,US,33,NaN,84.0,...,NaN,9.0,6.0,EWR,PIT,319.0,NaN,NaN,False,0
3,1990-01-04,4,1542.0,1540,1710.0,1701,US,33,NaN,88.0,...,NaN,9.0,2.0,EWR,PIT,319.0,NaN,NaN,False,0
4,1990-01-05,5,1549.0,1540,1706.0,1701,US,33,NaN,77.0,...,NaN,5.0,9.0,EWR,PIT,319.0,NaN,NaN,False,0
5,1990-01-06,6,1539.0,1540,1653.0,1701,US,33,NaN,74.0,...,NaN,-8.0,-1.0,EWR,PIT,319.0,NaN,NaN,False,0
6,1990-01-07,7,1553.0,1540,1713.0,1701,US,33,NaN,80.0,...,NaN,12.0,13.0,EWR,PIT,319.0,NaN,NaN,False,0
7,1990-01-08,1,1543.0,1540,1656.0,1701,US,33,NaN,73.0,...,NaN,-5.0,3.0,EWR,PIT,319.0,NaN,NaN,False,0
8,1990-01-09,2,1540.0,1540,1704.0,1701,US,33,NaN,84.0,...,NaN,3.0,0.0,EWR,PIT,319.0,NaN,NaN,False,0
9,1990-01-10,3,1608.0,1540,1740.0,1701,US,33,NaN,92.0,...,NaN,39.0,28.0,EWR,PIT,319.0,NaN,NaN,False,0


In [11]:
# total non-canceled flights per airport
airports = ddf.Origin.unique().compute()

for airport in airports:
    no = len(ddf[(ddf.Origin == airport) & ddf.Cancelled == False].compute())
    print(f'no of flights from {airport}: {no} ')

no of flights from EWR: 2577212 
no of flights from LGA: 2582739 
no of flights from JFK: 2604794 


In [13]:
# Average departure delay
for airport in airports:
    avg_delay = ddf[ddf.Origin == airport]["DepDelay"].mean().compute()
    print(f'average departure delay from {airport}: {avg_delay} ')

average departure delay from EWR: 10.295468607250333 
average departure delay from LGA: 7.431141565915709 
average departure delay from JFK: 10.351298909519874 


In [23]:
weekdays = np.arange(1,8)
weekday_dict = {1:'Monday', 2:'Tuesday',
                 3:'Wednesday', 4:'Thursday',
                 5:'Friday',6:'Saturday', 7:'Sunday'}
for airport in airports:
    max_delay_day = ddf[ddf.Origin == airport].groupby(['DayOfWeek'])["DepDelay"].mean().idxmax().compute()
    print(f' The day with the highest average delay at {airport} is {weekday_dict[max_delay_day]}')


 The day with the highest average delay at EWR is Friday
 The day with the highest average delay at LGA is Friday
 The day with the highest average delay at JFK is Friday
